In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.pylab import rcParams
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
rcParams['figure.figsize'] = 80,50
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score

In [16]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/labels.csv')

labelsTrain = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,3], inplace = True)
labelsTrain.columns = ["img_name"  , "gender"]
labelsTrain = labelsTrain.astype({'gender': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})
labelsTrain["gender"] = labelsTrain["gender"].replace(-1, 0)
#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/labels.csv')

labelsTest = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,3], inplace = True)
labelsTest.columns = ["img_name"  , "gender"]
labelsTest = labelsTest.astype({'gender': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})
labelsTest["gender"] = labelsTest["gender"].replace(-1, 0)

In [19]:
labelsTest

,img_name,gender
0,0.jpg,0
1,1.jpg,0
2,2.jpg,1
3,3.jpg,1
4,4.jpg,0
...,...,...
995,995.jpg,1
996,996.jpg,1
997,997.jpg,1
998,998.jpg,1


In [20]:
#Load training set################################
imagesTrain = np.zeros((5000, 218, 178))

for i in range(0,5000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/img/' + labelsTrain["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTrain[i] = image
    
imagesTrain = imagesTrain.reshape(5000,38804)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((1000, 218, 178))

for i in range(0,1000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/img/' + labelsTest["img_name"][i])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTest[i] = image
    
imagesTest = imagesTest.reshape(1000,38804)   

imagesTest = pd.DataFrame(imagesTest) 

In [21]:
#scale coversion
scaler = MinMaxScaler()

imagesTrain_scaled = scaler.fit_transform(imagesTrain)
imagesTest_scaled = scaler.transform(imagesTest)

In [22]:
#PCA conversion
pca = PCA(n_components = 200)

imagesTrain_pca = pca.fit_transform(imagesTrain_scaled)
imagesTest_pca = pca.transform(imagesTest_scaled)

imagesTrain_pca = pd.DataFrame(imagesTrain_pca) 
imagesTest_pca = pd.DataFrame(imagesTest_pca) 

In [23]:
pca.explained_variance_ratio_.sum()

0.9251020089592316

In [24]:
x_train = pd.DataFrame(imagesTrain_pca) 
x_test = pd.DataFrame(imagesTest_pca)
x_train = pd.concat([labelsTrain,x_train],axis=1, join='inner')

In [58]:
#logistic regression#######
model = LogisticRegression(solver='lbfgs', max_iter=311)
model.fit(x_train.iloc[:,2:], labelsTrain['gender'])
model.score(x_test,labelsTest['gender'])

0.877

In [26]:
x_train

,img_name,gender,0,1,2,3,4,5,6,7,...,190,191,192,193,194,195,196,197,198,199
0,0.jpg,0,26.669027,-3.010113,0.366209,-12.013860,-10.728321,4.044433,26.860383,5.570333,...,0.902716,0.441923,0.468601,1.498549,1.750221,-1.245941,-1.919863,0.659561,0.219470,-0.899978
1,1.jpg,0,-8.542702,20.497358,1.708910,-5.413798,-4.178639,1.759209,-6.362811,-13.287253,...,1.369914,-0.010996,-0.208139,0.584677,1.031834,-1.287013,0.061352,0.789740,-0.779490,0.596804
2,2.jpg,1,24.068655,-4.079156,-13.739504,-5.215119,-2.575907,13.642572,-9.142209,-18.403577,...,-0.930543,0.623297,-1.142276,0.188424,-0.880364,0.095365,0.010980,-0.911780,0.198082,0.165568
3,3.jpg,0,-40.542844,2.332370,25.871265,0.418673,0.756152,-7.282323,-14.998997,-0.572475,...,-0.904434,0.882680,-1.034054,0.465936,-0.481130,-1.798752,-0.254367,-1.313068,-0.653731,-0.706440
4,4.jpg,0,38.514245,7.115160,-14.013524,1.305666,-2.659288,3.774649,14.906124,4.406511,...,0.857419,-1.258443,0.392786,-0.229580,-0.630214,-0.029228,-0.037091,-1.130871,-0.127719,0.593694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995.jpg,1,-34.511117,24.379525,-10.107009,-32.831967,-7.765312,4.406494,-4.740041,-10.193578,...,-0.206444,-2.032998,-0.684923,0.547787,-0.340280,1.642273,2.407362,-0.813531,0.021720,0.482732
4996,4996.jpg,1,44.174510,-12.413210,-13.370676,-10.437810,5.773864,-15.869406,1.116529,-13.759635,...,0.568919,-1.304101,-0.106368,1.760126,0.130814,0.405541,-0.354993,1.960092,-0.568398,-0.164714
4997,4997.jpg,1,1.933570,2.291065,2.404113,-5.049897,-4.174332,-9.210949,2.470365,-5.281703,...,0.221205,-0.136679,0.499261,-0.553010,-0.768233,-0.563862,-0.410177,-0.367470,0.304270,0.460175
4998,4998.jpg,1,-24.263832,0.021518,0.488566,2.250925,2.302149,-7.468177,0.250570,-0.719291,...,0.297209,-0.118252,-0.378805,-0.869504,-0.068531,0.360388,0.101908,0.198435,-0.071674,-0.331247


In [27]:
# #decision tree#####
# model = tree.DecisionTreeClassifier(max_depth=5 ,criterion = 'gini')
# model.fit(imagesTrain_pca,labelsTrain['gender'])
# model.score(imagesTest_pca,labelsTest['gender'])

In [61]:
#logistic regression bagged ####### 
for i in range(680,730,10):
    samples=i
    bag1 = x_train.sample(samples,replace = True)
    model1 = LogisticRegression(solver='lbfgs', max_iter=200)
    model1.fit(bag1.iloc[:,2:],bag1['gender'])


    bag2 = x_train.sample(samples,replace = True)
    model2 = LogisticRegression(solver='lbfgs', max_iter=200)
    model2.fit(bag2.iloc[:,2:],bag2['gender'])

    bag3 = x_train.sample(samples,replace = True)
    model3 = LogisticRegression(solver='lbfgs', max_iter=200)
    model3.fit(bag3.iloc[:,2:],bag3['gender'])

    bag4 = x_train.sample(samples,replace = True)
    model4 = LogisticRegression(solver='lbfgs', max_iter=200)
    model4.fit(bag4.iloc[:,2:],bag4['gender'])    

    bag5 = x_train.sample(samples,replace = True)
    model5 = LogisticRegression(solver='lbfgs', max_iter=200)
    model5.fit(bag5.iloc[:,2:],bag5['gender'])

    bag6 = x_train.sample(samples,replace = True)
    model6 = LogisticRegression(solver='lbfgs', max_iter=200)
    model6.fit(bag6.iloc[:,2:],bag6['gender'])

    bag7 = x_train.sample(samples,replace = True)
    model7 = LogisticRegression(solver='lbfgs', max_iter=200)
    model7.fit(bag7.iloc[:,2:],bag7['gender'])

    a1 = model1.predict(x_test)  
    a2 = model2.predict(x_test) 
    a3 = model3.predict(x_test) 
    a4 = model4.predict(x_test) 
    a5 = model5.predict(x_test)
    a6 = model6.predict(x_test)
    a7 = model7.predict(x_test)
    predictions = []
    for i in range(0,1000):
        p = np.argmax(np.bincount([a1[i],a2[i],a3[i],a4[i],a5[i],a6[i],a7[i]]))
        predictions.append(p)
    print(accuracy_score(predictions,labelsTest['gender']))

C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

0.859


C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

0.852


C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

...

C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

0.863


C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

0.85


C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st